In [4]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import requests   
import time
import pandas as pd

org_url = "https://www.glassdoor.com"
main_url = "https://www.glassdoor.com/Reviews/J-P-Morgan-Reviews-E145.htm"

%matplotlib inline

# ---------------- Set up web scraper ---------------------

In [5]:
def getAndParseURL( url ):
#     time.sleep(5)
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    soup = BeautifulSoup(webpage, 'html.parser')
    return soup

# Get glassdoor.com -> J.P Morgan -> Employee reviews page urls

In [6]:
pages_url = [main_url]

# It could be easily observed that each page consists of about 10 reviews. If we would like the latest 1000 reviews, 
# it should be a reasonable idea to set the page limit as 200.
PAGE_LIMIT = 200

for i in range(2,PAGE_LIMIT):
    pages_url.append(org_url + "/Reviews/J-P-Morgan-Reviews-E145_P{}.htm".format(i))


# Get individual review urls


In [7]:
def get_reviews_url(page_url):
    reviews_url = set()

    soup = getAndParseURL(page_url)
    for x in soup.find_all('a',{'class':'reviewLink'}):
        if 'href' in x.attrs:
            reviews_url.add(x.get('href'))
    return reviews_url

# soup.find('a', {'class':'reviewLink'}).get('href')
# soup.find('div',{'class':'col-sm-4'}).span.get_text()

# Make sure that the pages we are about to visit are all valid

In [8]:
valid_status_code = 403
for url in pages_url[::30]:
    r = requests.get(url)          
    print("Page {0} is valid:".format(url), r.status_code==valid_status_code)


Page https://www.glassdoor.com/Reviews/J-P-Morgan-Reviews-E145.htm is valid: True
Page https://www.glassdoor.com/Reviews/J-P-Morgan-Reviews-E145_P31.htm is valid: True
Page https://www.glassdoor.com/Reviews/J-P-Morgan-Reviews-E145_P61.htm is valid: True
Page https://www.glassdoor.com/Reviews/J-P-Morgan-Reviews-E145_P91.htm is valid: True
Page https://www.glassdoor.com/Reviews/J-P-Morgan-Reviews-E145_P121.htm is valid: True
Page https://www.glassdoor.com/Reviews/J-P-Morgan-Reviews-E145_P151.htm is valid: True
Page https://www.glassdoor.com/Reviews/J-P-Morgan-Reviews-E145_P181.htm is valid: True


# Collection individual reviews. Including 'review title', 'review content', 'feedback', 'employee's status', 'working experience'.'working locations'

In [9]:
reviews_title = []
review = []
location = []
status = []
feedback = []
experience = []

counter = -1

urls = pages_url[:105]

for page_url in urls:
#     The process could take long, so we set up an indicator to remind us the progress of the programme.
    counter += 1
    print(str(100*counter/len(urls))+"% Finished")
    try:
        reviews_url = get_reviews_url(page_url)
    except:
        print(page_url)
        continue
    
    for review_url in reviews_url:
#         print(org_url+review_url)
        soup = getAndParseURL(org_url+review_url)
    
#     Add reviewer's job title
        i = soup.find('span',{'class':'authorJobTitle middle reviewer'})
        if i:
            j = i.get_text()
            status.append(j)
        else:
            status.append(None)

    
#     Add review's title
        i = soup.find('h2',{'class':'h2 summary strong mt-0 mb-xsm'})
        if i:
            j = i.find_next('span').get_text()
            reviews_title.append(j)
#             print(j)
        else:
            reviews_title.append(None)
    

#     Add reviewer's feedback 
        i = soup.find('div',{'class':'row reviewBodyCell recommends'})
        if i:
            temp = []
            for j in i.find_all('span'):
                temp.append(j.get_text())
            feedback.append(temp)
        else:
            feedback.append([None])

            
#     Add pros and cons reviews
    
        i = soup.find('div', {'class':'col-sm-11 pl-sm-lg mx-0'})
        if i:
            temp = []
            for j in i.find_all('p',{'class':'strong'}):
                temp.append(j.find_next('p').get_text())
            review.append(temp)
#             print(temp)
        else:
            review.append([None])
            
#     Add location of the reviewer
        i = soup.find('span',{'class':'authorLocation'})
        if i:
            location.append(i.text)
        else:
            location.append(None)
             
#     Add experience
        i = soup.find('p',{'class':'mainText mb-0'})
        if i:
            experience.append(i.text)
        else:
            experience.append(None)


0.0% Finished
0.9523809523809523% Finished
1.9047619047619047% Finished
2.857142857142857% Finished
3.8095238095238093% Finished
4.761904761904762% Finished
5.714285714285714% Finished


KeyboardInterrupt: 

# Put aside the collected data into catagories

In [ ]:
df = pd.DataFrame({'Reviews_Title':reviews_title,'review':review, 
                   'experience':experience,'Location':location,
                   'Status':status,'Feedback':feedback})

# Get Years of experience
slicing = df['experience'].str.index("Morgan")
df['Experience'] = df['experience'].str.split('Morgan').str[1]

# Split reviews into positive and negative reviews
reviews = df['review'].apply(pd.Series)
reviews = reviews.rename(columns={0:'Positive Review',1:'Negative Review',2:'Suggestions'})
df = pd.concat([df[:],reviews[:]],axis=1).drop(['review','experience'],axis = 1)

# Rearrange the order of columns
columns = ['Location','Experience','Status','Feedback','Reviews_Title','Positive Review','Negative Review','Suggestions']
df = df[columns]


# Export the raw data

In [ ]:
df.to_excel('Raw_data.xlsx')

# Clean and orgnize the raw data

In [ ]:
Employee_Review = pd.read_excel('Raw_data.xlsx')

In [ ]:
from ast import literal_eval
Employee_Review['Feedback'] = Employee_Review['Feedback'].apply(literal_eval)
Employee_Review = Employee_Review.loc[Employee_Review['Feedback'].map(len)==3]
Employee_Review[['Recommandation','Outlook','CEO Opinion']] = Employee_Review['Feedback'].apply(pd.Series)
Employee_Review = Employee_Review.drop(columns=['Feedback'],axis=1)

# A preview of the clean data

In [ ]:
Employee_Review.head()

# Export the final dataset as excel spreadsheet

In [ ]:
# Export to excel spread sheet
Employee_Review.to_excel("Employee_Review.xlsx")